In [ ]:
!pip install simpletransformers
# !pip install pysentimiento
# !pip install --upgrade accelerate

In [ ]:
!pip uninstall transformers -y

Found existing installation: transformers 4.29.2
Uninstalling transformers-4.29.2:
  Successfully uninstalled transformers-4.29.2


In [ ]:
!pip install transformers==4.28.0

In [ ]:
import pandas as pd
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from simpletransformers.classification import ClassificationModel, ClassificationArgs
# from pysentimiento import create_analyzer
# analyzer = create_analyzer(task="sentiment", lang="es")

In [ ]:
from pysentimiento import create_analyzer
analyzer = create_analyzer(task="sentiment", lang="es")

## DATA MANIPULATION

### Hate Detection - Marathi

In [ ]:
hate_mr = pd.read_excel('/content/hate_bin_test.xlsx')

In [ ]:
hate_mr.head(5)

,text,label
0,@GajuNaik20 @NitinRaut_INC अबे झाटु लालबहादूर ...,HOF
1,@khadaksingh_ अंड नसलेल्या भक्तांना मोदी फायद्...,HOF
2,@BhatkhalkarA अरे माठ्या त्यांच्याकडून काहीच श...,HOF
3,हे हिंदूंचे सरकार नव्हे तर लांड्यांचे सरकार ! ...,HOF
4,@_prashantkadam कदम रडू नको तुझा मालक कोर्टात ...,HOF


In [ ]:
hate_mr['label'].value_counts()

HOF    1875
NOT    1875
Name: label, dtype: int64

In [ ]:
unique_labels = hate_mr['label'].unique()
print(unique_labels)

['HOF' 'NOT']


In [ ]:
hate_mr_final = pd.DataFrame()  

for label in unique_labels:
    label_rows = hate_mr[hate_mr['label'] == label]  
    random_rows = label_rows.sample(n=50)  
    hate_mr_final = hate_mr_final.append(random_rows) 

<ipython-input-171-289813312247>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hate_mr_final = hate_mr_final.append(random_rows)
<ipython-input-171-289813312247>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hate_mr_final = hate_mr_final.append(random_rows)


In [ ]:
hate_mr_final = hate_mr_final.reset_index(drop = True)

In [ ]:
hate_mr_final.head(3)

,text,label
0,@MChormal @gandolenitin मर्दाच्या अवलादी सत्ते...,HOF
1,@INCMaharashtra कुठली शाळा शिकलेले आहे बरं... ...,HOF
2,ईदच्या पवित्र दिवशी जम्मू आणि काश्मिरात राष्ट्...,HOF


In [ ]:
hate_mr_final['label'].value_counts()

HOF    50
NOT    50
Name: label, dtype: int64

In [ ]:
hate_mr_final = hate_mr_final.sample(frac=1).reset_index(drop=True)

In [ ]:
hate_mr_final.to_csv('hate_mr_final.csv')

### Sentiement Analysis - Marathi

In [ ]:
sentiment_mr = pd.read_csv('/content/tweets-test.csv')

In [ ]:
sentiment_mr.head(5)

,tweet,label
0,#BetiBachao चा फक्त नारा देऊन उपयोग नाही. महि...,1
1,पेट्रोल आणि डिझेलवर एक रूपया अधिभार लावल्यान...,-1
2,"लूट झूट का राज भागावो, \nरोजगार और विकास ला...",1
3,महाराष्ट्र विकास आघाडी सरकारच्या मंत्रिमंडळ ...,1
4,पिंपरीत अ‍ॅथलेटिक्स आणि विविध क्षेत्रांतल्या...,1


In [ ]:
sentiment_mr['label'].value_counts()

 1    750
-1    750
 0    750
Name: label, dtype: int64

In [ ]:
unique_labels = [1, -1]
print(unique_labels)

[1, -1]


In [ ]:
sentiment_mr_final = pd.DataFrame()  

for label in unique_labels:
    label_rows = sentiment_mr[sentiment_mr['label'] == label]  
    random_rows = label_rows.sample(n=50)  
    sentiment_mr_final = sentiment_mr_final.append(random_rows) 

<ipython-input-181-6e0c1cb1cc2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_mr_final = sentiment_mr_final.append(random_rows)
<ipython-input-181-6e0c1cb1cc2f>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_mr_final = sentiment_mr_final.append(random_rows)


In [ ]:
sentiment_mr_final = sentiment_mr_final.reset_index(drop = True)

In [ ]:
sentiment_mr_final.head(3)

,tweet,label
0,मनःपूर्वक आभार !\n,1
1,मनोहर जोशी या कार्यक्रमाला उपस्थित होते. ते ...,1
2,"देशाचं स्वातंत्र्य, एकता, अखंडता, सार्वभौमता...",1


In [ ]:
sentiment_mr_final['label'].value_counts()

 1    50
-1    50
Name: label, dtype: int64

In [ ]:
sentiment_mr_final = sentiment_mr_final.sample(frac=1).reset_index(drop=True)

In [ ]:
sentiment_mr_final.to_csv('sentiment_mr_final.csv')

### NER Marathi

In [ ]:
words = list()
tags = list()
sentence_id = list()
temp_id = 0
sentence_id_temp = list()
tags_temp = list()
words_temp = list()

with open('/content/test_noniob.txt', 'r', encoding='utf-8') as file:
    for line in file:
      line = line.strip().split('\t')
      if line[2] == 'sentence_id':
        continue
      if int(float(line[2])) != temp_id:
        words.append(words_temp)
        tags.append(tags_temp)
        sentence_id.append(sentence_id_temp)
        sentence_id_temp = list()
        tags_temp = list()
        words_temp = list()
        temp_id +=1


      words_temp.append(line[0])
      tags_temp.append(line[1])
      sentence_id_temp.append(int(float(line[2])))

In [ ]:
len(words) == len(tags) == len(sentence_id)

True

In [ ]:
print(len(words))
print(len(tags))
print(len(sentence_id))

2001
2001
2001


In [ ]:
words[:5]

[[],
 ['जैश-ए-मोहम्मद',
  'या',
  'दहशतवादी',
  'संघटनेने',
  'या',
  'हल्ल्याची',
  'जबाबदारी',
  'घेतली',
  'होती '],
 ['मुंबईत',
  'पाहणी',
  'करण्यासाठी',
  'पहिल्यादांचा',
  'ड्रोनचा',
  'वापर',
  'करण्यात',
  'येणार',
  'आहे '],
 ['सेल्फीसाठी',
  'या',
  'फोनमध्ये',
  '२९',
  'मेगापिक्सलचा',
  'कॅमेरा',
  'दिला',
  'आहे '],
 ['आशियाई',
  'देशांमध्येही',
  'फिलिपिनी',
  'नर्सेस',
  'खालोखाल',
  'भारतीय',
  'नर्सेसना',
  'पसंती',
  'मिळत',
  'आहे ']]

In [ ]:
words = words[1:]
tags = tags[1:]
sentence_id = sentence_id[1:]

In [ ]:
for i in range(len(words)):
  words[i] = " ".join(words[i])

for i in range(len(tags)):
  tags[i] = " ".join(tags[i])

In [ ]:
words[:5]

['जैश-ए-मोहम्मद या दहशतवादी संघटनेने या हल्ल्याची जबाबदारी घेतली होती ',
 'मुंबईत पाहणी करण्यासाठी पहिल्यादांचा ड्रोनचा वापर करण्यात येणार आहे ',
 'सेल्फीसाठी या फोनमध्ये २९ मेगापिक्सलचा कॅमेरा दिला आहे ',
 'आशियाई देशांमध्येही फिलिपिनी नर्सेस खालोखाल भारतीय नर्सेसना पसंती मिळत आहे ',
 'असे असतानाही दुहेरी हत्याकांड घडले आहे ']

In [ ]:
tags[:5]

['NEO O O O O O O O O',
 'NEL O O O O O O O O',
 'O O O NEM O O O O',
 'O O O O O O O O O O',
 'O O O O O O']

In [ ]:
df_ner = pd.DataFrame({'Words': words, 'Tags': tags})

In [ ]:
df_ner.head(5)

,Words,Tags
0,जैश-ए-मोहम्मद या दहशतवादी संघटनेने या हल्ल्याच...,NEO O O O O O O O O
1,मुंबईत पाहणी करण्यासाठी पहिल्यादांचा ड्रोनचा व...,NEL O O O O O O O O
2,सेल्फीसाठी या फोनमध्ये २९ मेगापिक्सलचा कॅमेरा ...,O O O NEM O O O O
3,आशियाई देशांमध्येही फिलिपिनी नर्सेस खालोखाल भा...,O O O O O O O O O O
4,असे असतानाही दुहेरी हत्याकांड घडले आहे,O O O O O O


In [ ]:
df_ner = df_ner.sample(n=100) 
df_ner = df_ner.reset_index(drop = True)

In [ ]:
df_ner

,Words,Tags
0,यावेळी प्रत्येकानेच सोशल मीडियावर त्यांच्या बा...,O O O O O O O O O
1,लोन मोरेटोरिम बाबत दास यांनी पत्रकार परिषदेत क...,O O O NEP O O O O O O
2,काही दिवसांपूर्वी हसीला काही जणांनी धमकी दिली ...,O O NEP O O O O O
3,धोनीचा लुक कायम सोशल मीडियावर चर्चेत असतो,NEP O O O O O O
4,अनेकांना मिळाले इलेक्शन गिफ्ट निवडणुकीच्या काळ...,O O O O O O O O O O O O
...,...,...
95,चित्रदुर्ग जिल्ह्यातील हिरियूरजवळच ही दुर्घटना...,NEL O NEL O O O
96,हे देखील वाचानोकरदारांसाठी आनंदाची बातमी एका व...,O O O O O NEM O O O
97,प्रत्येक दुकानदारास त्याची सीमा निर्धारित आहे,O O O O O O
98,तर राज्यातील करोना मृत्यूचा दर ३,O O O O O NEM


In [ ]:
df_ner.to_csv('ner_mr_final.csv', index=False)

### Hate Detection Spanish

In [ ]:
text_list = list()
label_list = list()

with open("/content/Hate labeled_corpus_6K.txt", "r") as file:
    for line in file:
      split_text = line.split("||;")
      if len(split_text) == 3:
        text = split_text[1]
        label = split_text[2]
    
      text_list.append(text)
      label_list.append(label)

label_list = [s.strip() for s in label_list]
label_list = [int(s) for s in label_list]

In [ ]:
df_hat_es = pd.DataFrame({'Text': text_list, 'Tags': label_list})

In [ ]:
df_hat_es

,Text,Tags
0,Ismael es egocentrico porque se vuelve loca si...,0
1,..ya tardaba en salir quien pronunciase nombre...,0
2,"(Esto no es un discurso político y razonado, o...",0
3,"Muy despreciados,siiii,pero todos vestidos de ...",1
4,marica explicame porque a veces no te entiendo...,1
...,...,...
5995,"Sea independentista o constitucionalista, la d...",0
5996,@Bernithedude Llave de gobierno de qué si hay ...,0
5997,@13Pardis @guenhwyvarblack Los carlinos son un...,0
5998,Aplausos en el partido de Colau cuando anuncia...,0


In [ ]:
unique_labels = df_hat_es['Tags'].unique()
print(unique_labels)

[0 1]


In [ ]:
df_hat_es_final = pd.DataFrame()  

for label in unique_labels:
    label_rows = df_hat_es[df_hat_es['Tags'] == label]  
    random_rows = label_rows.sample(n=50)  
    df_hat_es_final = df_hat_es_final.append(random_rows) 

<ipython-input-191-1950b9bb14d7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hat_es_final = df_hat_es_final.append(random_rows)
<ipython-input-191-1950b9bb14d7>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_hat_es_final = df_hat_es_final.append(random_rows)


In [ ]:
df_hat_es_final = df_hat_es_final.reset_index(drop = True)

In [ ]:
df_hat_es_final

,Text,Tags
0,"Y mientras tanto, en Venezuela, continúa la VE...",0
1,"En escaños gana el independentismo, pero en nú...",0
2,"Es que la peña es subnormal, publicidad gratis.;",0
3,Metiendo bulla: La señora independentista maqu...,0
4,"@jcamposasensi Coincido, Jorge. Ambos son resp...",0
...,...,...
95,Dicho esto entiendo que linchen a Antonio Maes...,1
96,"@ymouled Cembrero es un gilipollas, racista y ...",1
97,No haber votado a un tarado que salía en todos...,1
98,Si fuera independentista no votaría jamás a la...,1


In [ ]:
df_hat_es_final['Tags'].value_counts()

0    50
1    50
Name: Tags, dtype: int64

In [ ]:
df_hat_es_final = df_hat_es_final.sample(frac=1).reset_index(drop=True)

In [ ]:
df_hat_es_final.to_csv('hate_es_final.csv')

### Sentiment Analysis Spanish

In [ ]:
import requests

url = "http://tass.sepln.org/tass_data/dataset/intertass-ES-development-tagged.xml"  # Replace with the URL of the XML file
file_path = "/content/Sentiment_es.xml"   # Replace with the desired local file path

response = requests.get(url)
response.raise_for_status()  # Check for any request errors

with open(file_path, "wb") as file:
    file.write(response.content)

print("XML file downloaded successfully.")


XML file downloaded successfully.


In [ ]:
import xml.etree.ElementTree as ET

file_path = "/content/Sentiment_es.xml"  # Replace with the path to your XML file

# Initialize empty lists
text = []
label = []

# Parse the XML file
tree = ET.parse(file_path)
root = tree.getroot()

# Iterate over each 'tweet' element in the XML
for tweet_elem in root.findall('tweet'):
    content_elem = tweet_elem.find('content')
    value_elem = tweet_elem.find('sentiment/polarity/value')
    
    # Extract the content and value
    if content_elem is not None and value_elem is not None:
        content = content_elem.text.strip()
        value = value_elem.text.strip()
        
        # Append the extracted content and value to the lists
        text.append(content)
        label.append(value)

# Print the extracted text and label
print("Text:", text)
print("Label:", label)

Text: ['@noseashetero 1000/10 de verdad a ti que voy a decir petarda que te quiero más que a mí mismo  ✨', '@piscolabisaereo @HistoriaNG @SPosteguillo las tengo pero aún no las he leído. Caerán prontito', 'Al final han sido 3h  Bueno, mañana tengo fiesta así que.. No me quejo', '@Jorge_Ruiz14 yo no tengo tiempo para esas cosas ahora mismo', '@_MissChaotic_ ves ese brillo? es un coso que hace que se sepan a kk', 'Tengo una perrina adorable... Sabéis que me acompaña hasta la a habitación cuando me voy a dormir?', '@juankipua Es que en el Ojeando el año pasado tampoco, tiene muchas canciones ya jajajajaja', 'Bueno, estoy en la batalla final del Conquista y ya después me faltaría Revelación', '@CondeDuqueMAD ¿mañana sábado 31? En que día vives?  Mañana es miércoles 31.', '@agcasti y el caminante ante el mar de niebla, uno de mis cuadros favoritos en portada', '@vitojph ¡sí! Y me ha encantado  ¿Tú ya las visto? ¿Cuándo comentamos?', '@aladroqe Se me olvidaban los grandes @2qblog y @estapill

In [ ]:
len(text) == len(label)

True

In [ ]:
df_sentiment_es = pd.DataFrame({'Text': text, 'Tags': label})

In [ ]:
df_sentiment_es.head(3)

,Text,Tags
0,@noseashetero 1000/10 de verdad a ti que voy a...,P
1,@piscolabisaereo @HistoriaNG @SPosteguillo las...,P
2,"Al final han sido 3h Bueno, mañana tengo fies...",P


In [ ]:
df_sentiment_es['Tags'].value_counts()

N       219
P       156
NEU      69
NONE     62
Name: Tags, dtype: int64

In [ ]:
df_sentiment_es = df_sentiment_es[df_sentiment_es['Tags'] != 'NONE']

In [ ]:
df_sentiment_es['Tags'].value_counts()

N      219
P      156
NEU     69
Name: Tags, dtype: int64

In [ ]:
unique_labels = df_sentiment_es['Tags'].unique()
print(unique_labels)

['P' 'N' 'NEU']


In [ ]:
sentiment_es_final = pd.DataFrame()  

for label in unique_labels:
    label_rows = df_sentiment_es[df_sentiment_es['Tags'] == label]  
    random_rows = label_rows.sample(n=33)  
    sentiment_es_final = sentiment_es_final.append(random_rows) 

<ipython-input-206-a0182ff363d3>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_es_final = sentiment_es_final.append(random_rows)
<ipython-input-206-a0182ff363d3>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_es_final = sentiment_es_final.append(random_rows)
<ipython-input-206-a0182ff363d3>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sentiment_es_final = sentiment_es_final.append(random_rows)


In [ ]:
sentiment_es_final = sentiment_es_final.reset_index(drop = True)

In [ ]:
sentiment_es_final.head(3)

,Text,Tags
0,@JulenReal Anda que no nos hemos subido ahí tu...,P
1,Ya decía yo que lo bueno me iba a llegar tarde...,P
2,"@pmklose Le conocí en 2011, me pareció intelig...",P


In [ ]:
sentiment_es_final['Tags'].value_counts()

P      33
N      33
NEU    33
Name: Tags, dtype: int64

In [ ]:
sentiment_es_final = sentiment_es_final.sample(frac=1).reset_index(drop=True)

In [ ]:
sentiment_es_final.to_csv('sentiment_es_final.csv')

## EVALUATION

In [ ]:
df = pd.read_csv('/content/CSE-291-PROJECT - FINAL.csv')

In [ ]:
df.head(3)

In [ ]:
def calculate_metrics(real, pred, model):
    f1_micro = f1_score(real, pred, average='micro')
    f1_macro = f1_score(real, pred, average='macro')
    precision = precision_score(real, pred, average='weighted')
    recall = recall_score(real, pred, average='weighted')
    accuracy = accuracy_score(real, pred)

    print("F1 Micro", model, ": ", f1_micro)
    print("F1 Macro:", model, ": ", f1_macro)
    print("Precision:", model, ": ", precision)
    print("Recall:", model, ": ", recall)
    print("Accuracy:", model, ": ", accuracy)

### Hate Detection - Marathi - Hate = 1, NotHate = 0

In [ ]:
df_real_hate_mr = pd.read_csv('/content/hate_mr_final.csv')

In [ ]:
df_real_hate_mr['label'].value_counts()

HOF    50
NOT    50
Name: label, dtype: int64

In [ ]:
label_to_num = {'HOF': 1, 'NOT': 0}
num_to_label = {0: 'NOT', 1: 'HOF'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'HOF': 1, 'NOT': 0}
 
{0: 'NOT', 1: 'HOF'}


In [ ]:
df_real_hate_mr['tokeinzed_label'] = df_real_hate_mr['label'].apply(lambda x: label_to_num[x])

In [ ]:
df_real_hate_mr['tokeinzed_label']

0     1
1     1
2     0
3     1
4     0
     ..
95    1
96    0
97    0
98    0
99    0
Name: tokeinzed_label, Length: 100, dtype: int64

#### ChatGPT

In [ ]:
label_to_num = {cuisine: i for i, cuisine in enumerate(df['hate_mr_ChatGPT'].unique())}      
num_to_label = {i: cuisine for cuisine, i in label_to_num.items()}

In [ ]:
print(label_to_num)
print(" ")
print(label_to_num)

{'not hate': 0, 'hate': 1}
 
{'not hate': 0, 'hate': 1}


In [ ]:
df['tokenized_hate_mr_ChatGPT'] = df['hate_mr_ChatGPT'].apply(lambda x: label_to_num[x])

In [ ]:
df['tokenized_hate_mr_ChatGPT']

0     0
1     1
2     1
3     1
4     0
     ..
95    1
96    0
97    1
98    0
99    0
Name: tokenized_hate_mr_ChatGPT, Length: 100, dtype: int64

In [ ]:
calculate_metrics(df_real_hate_mr['tokeinzed_label'], df['tokenized_hate_mr_ChatGPT'], 'ChatGPT')

F1 Micro ChatGPT :  0.78
F1 Macro: ChatGPT :  0.7767857142857144
Precision: ChatGPT :  0.7971137521222411
Recall: ChatGPT :  0.78
Accuracy: ChatGPT :  0.78


#### Perplexity AI

In [ ]:
def remove_indexes(df, column_name):
    pattern = r'^\d+[\s-]+'
    df[column_name] = df[column_name].str.replace(pattern, '', regex=True)
    return df

In [ ]:
df['hate_mr_Perplexity AI']

0          0 hate
1          1 hate
2          2 hate
3      3 not hate
4      4 not hate
         ...     
95        95 hate
96    96 not hate
97        97 hate
98    98 not hate
99    99 not hate
Name: hate_mr_Perplexity AI, Length: 100, dtype: object

In [ ]:
df = remove_indexes(df, 'hate_mr_Perplexity AI')

In [ ]:
df['hate_mr_Perplexity AI']

0         hate
1         hate
2         hate
3     not hate
4     not hate
        ...   
95        hate
96    not hate
97        hate
98    not hate
99    not hate
Name: hate_mr_Perplexity AI, Length: 100, dtype: object

In [ ]:
df['hate_mr_Perplexity AI'].value_counts()

hate        50
not hate    50
Name: hate_mr_Perplexity AI, dtype: int64

In [ ]:
label_to_num = {'hate': 1, 'not hate': 0}   
num_to_label = {1: 'hate', 0: 'not hate'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'hate': 1, 'not hate': 0}
 
{1: 'hate', 0: 'not hate'}


In [ ]:
df['tokenized_hate_mr_Perplexity_AI'] = df['hate_mr_Perplexity AI'].apply(lambda x: label_to_num[x])

In [ ]:
df['tokenized_hate_mr_Perplexity_AI']

0     1
1     1
2     1
3     0
4     0
     ..
95    1
96    0
97    1
98    0
99    0
Name: tokenized_hate_mr_Perplexity_AI, Length: 100, dtype: int64

In [ ]:
calculate_metrics(df_real_hate_mr['tokeinzed_label'], df['tokenized_hate_mr_Perplexity_AI'], 'Perplexity AI')

F1 Micro Perplexity AI :  0.7
F1 Macro: Perplexity AI :  0.7
Precision: Perplexity AI :  0.7
Recall: Perplexity AI :  0.7
Accuracy: Perplexity AI :  0.7


#### ChatSonic

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('\d+', '', regex=True)
  df[column_name] = df[column_name].str.replace(':', '')
  df[column_name] = df[column_name].str.replace('speech', '')
  df[column_name] = df[column_name].str.replace('Speech', '')
  df[column_name] = df[column_name].str.replace('-', '')
  df[column_name] = df[column_name].str.replace('  ', '')
  df[column_name] = df[column_name].str.replace(' ', '')
  df[column_name] = df[column_name].str.replace('  ', '')
  df[column_name] = df[column_name].str.replace('Nonhate', 'NonHate')
  return df

In [ ]:
df['hate_mr_Chatsonic']

0     NonHate
1        Hate
2        Hate
3     NonHate
4     NonHate
       ...   
95       Hate
96    NonHate
97       Hate
98    NonHate
99    NonHate
Name: hate_mr_Chatsonic, Length: 100, dtype: object

In [ ]:
df = remove_substrings(df, 'hate_mr_Chatsonic')

In [ ]:
df['hate_mr_Chatsonic']

0     NonHate
1        Hate
2        Hate
3     NonHate
4     NonHate
       ...   
95       Hate
96    NonHate
97       Hate
98    NonHate
99    NonHate
Name: hate_mr_Chatsonic, Length: 100, dtype: object

In [ ]:
df['hate_mr_Chatsonic'].value_counts()

NonHate    58
Hate       42
Name: hate_mr_Chatsonic, dtype: int64

In [ ]:
label_to_num = {cuisine: i for i, cuisine in enumerate(df['hate_mr_Chatsonic'].unique())}      
num_to_label = {i: cuisine for cuisine, i in label_to_num.items()}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'NonHate': 0, 'Hate': 1}
 
{0: 'NonHate', 1: 'Hate'}


In [ ]:
df['tokenized_hate_mr_Chatsonic'] = df['hate_mr_Chatsonic'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_hate_mr['tokeinzed_label'], df['tokenized_hate_mr_Chatsonic'], 'ChatSonic')

F1 Micro ChatSonic :  0.68
F1 Macro: ChatSonic :  0.677938808373591
Precision: ChatSonic :  0.6847290640394089
Recall: ChatSonic :  0.68
Accuracy: ChatSonic :  0.68


#### MahaHate - L3Cube

In [ ]:
df_predict_hate_mr = df_real_hate_mr[['text']]

In [ ]:
df_predict_hate_mr

,text
0,शेजारणीची सुंदर पुच्ची - https://t.co/xYqE52MoT1
1,"दलिंदर आहे काँग्रेस ची मानसिकता, देशाला लागलेल..."
2,"सामूहिक बलात्कारानंतर अल्पवयीन मुलीची हत्या, ऊ..."
3,@INCMaharashtra कुठली शाळा शिकलेले आहे बरं... ...
4,झी चोवीस ‌वाले अर्धवट बातम्या देण्यात एक नंबर ...
...,...
95,ईदच्या पवित्र दिवशी जम्मू आणि काश्मिरात राष्ट्...
96,या मार्गावर धावणाऱ्या रेल्वे गाड्यांना हा प्रव...
97,@MaheshPatil_B मुळात महाराजांची विटंबणा करण्या...
98,लबडे परीवार बैलपोळा ऊत्सव !! https://t.co/X0W...


In [ ]:
model = ClassificationModel("bert", "l3cube-pune/mahahate-bert")

In [ ]:
predictions, raw_outputs = model.predict(df_predict_hate_mr['text'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
predictions

array([1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0])

In [ ]:
df_mahahate = pd.DataFrame(predictions, columns=['labels'])

In [ ]:
calculate_metrics(df_real_hate_mr['tokeinzed_label'], df_mahahate['labels'], 'MahaHate - L3Cube')

F1 Micro MahaHate - L3Cube :  0.8599999999999999
F1 Macro: MahaHate - L3Cube :  0.8599999999999999
Precision: MahaHate - L3Cube :  0.86
Recall: MahaHate - L3Cube :  0.86
Accuracy: MahaHate - L3Cube :  0.86


### Sentiment Analysis - Marathi - Positive(1), Negative(-1)

In [ ]:
df_real_sentiment_mr = pd.read_csv('/content/sentiment_mr_final.csv')

In [ ]:
df_real_sentiment_mr['label'].value_counts()

-1    50
 1    50
Name: label, dtype: int64

In [ ]:
df_real_sentiment_mr['label']

0    -1
1     1
2    -1
3     1
4     1
     ..
95   -1
96    1
97    1
98    1
99    1
Name: label, Length: 100, dtype: int64

#### ChatGPT

In [ ]:
df['sentiment_mr_ChatGPT'].value_counts()

Positive    55
Negative    35
negative     7
positive     3
Name: sentiment_mr_ChatGPT, dtype: int64

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('negative', 'Negative')
  df[column_name] = df[column_name].str.replace('positive', 'Positive')
  return df

In [ ]:
df = remove_substrings(df, 'sentiment_mr_ChatGPT')

In [ ]:
df['sentiment_mr_ChatGPT'].value_counts()

Positive    58
Negative    42
Name: sentiment_mr_ChatGPT, dtype: int64

In [ ]:
label_to_num = {'Negative': -1, 'Positive': 1}   
num_to_label = {-1: 'Negative', 1: 'Positive'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'Negative': -1, 'Positive': 1}
 
{-1: 'Negative', 1: 'Positive'}


In [ ]:
df['tokenized_sentiment_mr_ChatGPT'] = df['sentiment_mr_ChatGPT'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_sentiment_mr['label'], df['tokenized_sentiment_mr_ChatGPT'], 'ChatGPT')

F1 Micro ChatGPT :  0.8399999999999999
F1 Macro: ChatGPT :  0.8389694041867956
Precision: ChatGPT :  0.8489326765188835
Recall: ChatGPT :  0.84
Accuracy: ChatGPT :  0.84


#### Perplexity AI

In [ ]:
df['sentiment_mr_Perplexity AI']

0      0 - negative
1      1 - positive
2      2 - negative
3      3 - negative
4      4 - positive
          ...      
95    95 - negative
96    96 - positive
97    97 - positive
98    98 - negative
99    99 - positive
Name: sentiment_mr_Perplexity AI, Length: 100, dtype: object

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('\d+', '', regex=True)
  df[column_name] = df[column_name].str.replace(' - ', '')
  df[column_name] = df[column_name].str.replace('unclear', 'negative')
  return df

In [ ]:
df = remove_substrings(df, 'sentiment_mr_Perplexity AI')

In [ ]:
df['sentiment_mr_Perplexity AI'].value_counts()

positive    57
negative    43
Name: sentiment_mr_Perplexity AI, dtype: int64

In [ ]:
label_to_num = {'negative': -1, 'positive': 1}   
num_to_label = {-1: 'negative', 1: 'positive'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'negative': -1, 'positive': 1}
 
{-1: 'negative', 1: 'positive'}


In [ ]:
df['tokenized_sentiment_mr_Perplexity_AI'] = df['sentiment_mr_Perplexity AI'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_sentiment_mr['label'], df['tokenized_sentiment_mr_Perplexity_AI'], 'Perplexity AI')

F1 Micro Perplexity AI :  0.85
F1 Macro: Perplexity AI :  0.8492613807657521
Precision: Perplexity AI :  0.8569971440228477
Recall: Perplexity AI :  0.85
Accuracy: Perplexity AI :  0.85


#### ChatSonic

In [ ]:
df['sentiment_mr_Chatsonic']

0       0: Negative
1       1: Positive
2          Negative
3          Negative
4          Positive
          ...      
95    95 - Negative
96    96 - Positive
97    97 - Positive
98     98 - Neutral
99    99 - Positive
Name: sentiment_mr_Chatsonic, Length: 100, dtype: object

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('\d+', '', regex=True)
  df[column_name] = df[column_name].str.replace(' - ', '')
  df[column_name] = df[column_name].str.replace(': ', '')
  df[column_name] = df[column_name].str.replace('Neutral/Positive', 'Positive')
  df[column_name] = df[column_name].str.replace('Mixed (Neutral and Negative)', 'Negative')
  df[column_name] = df[column_name].str.replace(' Negative', 'Negative')
  df[column_name] = df[column_name].str.replace('Neutral', 'Positive')
  df[column_name] = df[column_name].str.replace(' Positive', 'Positive')
  df[column_name] = df[column_name].str.replace(' Mixed', 'Negative')
  df[column_name] = df[column_name].str.replace('Positive sentiment.', 'Positive')
  df[column_name] = df[column_name].str.replace('Mixed (Positive andNegative)', 'Negative')
  return df

In [ ]:
df = remove_substrings(df, 'sentiment_mr_Chatsonic')

<ipython-input-168-16bff9eb52df>:6: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] = df[column_name].str.replace('Mixed (Neutral and Negative)', 'Negative')
<ipython-input-168-16bff9eb52df>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] = df[column_name].str.replace('Positive sentiment.', 'Positive')
<ipython-input-168-16bff9eb52df>:12: FutureWarning: The default value of regex will change from True to False in a future version.
  df[column_name] = df[column_name].str.replace('Mixed (Positive andNegative)', 'Negative')


In [ ]:
df['sentiment_mr_Chatsonic'].unique()

array(['Negative', 'Positive', 'Mixed (Positive andNegative)'],
      dtype=object)

In [ ]:
label_to_num = {'Negative': -1, 'Positive': 1, 'Mixed (Positive andNegative)': -1}   
# num_to_label = {-1: 'negative', 1: 'positive'}

In [ ]:
print(label_to_num)
print(" ")
# print(num_to_label)

{'Negative': -1, 'Positive': 1, 'Mixed (Positive andNegative)': -1}
 


In [ ]:
df['tokenized_sentiment_mr_Chatsonic'] = df['sentiment_mr_Chatsonic'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_sentiment_mr['label'], df['tokenized_sentiment_mr_Chatsonic'], 'ChatSonic')

F1 Micro ChatSonic :  0.8399999999999999
F1 Macro: ChatSonic :  0.839421918908069
Precision: ChatSonic :  0.8449675324675325
Recall: ChatSonic :  0.84
Accuracy: ChatSonic :  0.84


#### MahaSentiment - L3Cube

In [ ]:
df_predict_sentiment_mr = df_real_sentiment_mr[['tweet']]
df_predict_sentiment_mr = df_predict_sentiment_mr.rename(columns={"tweet": "text"})

In [ ]:
df_predict_sentiment_mr

,text
0,शासन कर्जमाफीतून शिवाजी महाराजांच्या नावाखाल...
1,राष्ट्रीय स्वयंसेवक संघाचे चौथे सरसंघचालक प्...
2,आधुनिक शहरांची निर्मिती म्हणजे हजारो कोटी रु...
3,"आज राडी, बर्दापूर, घाटनांदुर, मांडवा, धर्मपु..."
4,या नोकरी महोत्सवात सुमारे पाच हजार बेरोजगारा...
...,...
95,सध्या सत्तेचा गैरवापर करून विरोधकांना त्रास ...
96,छत्रपती शिवाजी महाराजांचा देदीप्यमान पराक्रम...
97,भोकरदनच्या मातृशक्तीचा आशीर्वाद हा माझ्यासाठ...
98,पुणे जिल्ह्यातील भोसरी येथे आज झालेल्या ‘रोड...


In [ ]:
model = ClassificationModel("albert", "l3cube-pune/MarathiSentiment")

In [ ]:
predictions, raw_outputs = model.predict(df_predict_sentiment_mr['text'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
predictions

array([0, 2, 0, 2, 1, 0, 0, 0, 1, 2, 0, 0, 2, 2, 2, 0, 0, 0, 2, 1, 2, 1,
       0, 2, 0, 0, 1, 2, 2, 1, 0, 0, 0, 2, 0, 2, 1, 0, 0, 0, 0, 0, 0, 2,
       1, 0, 1, 1, 2, 2, 0, 0, 0, 0, 2, 1, 0, 2, 0, 0, 1, 2, 2, 2, 0, 0,
       0, 2, 0, 0, 1, 2, 2, 2, 0, 1, 0, 0, 2, 0, 2, 2, 2, 1, 2, 0, 1, 2,
       0, 1, 2, 2, 2, 0, 0, 0, 2, 1, 2, 2])

In [ ]:
df_mahasenti = pd.DataFrame(predictions, columns=['labels'])

In [ ]:
calculate_metrics(df_real_sentiment_mr['label'], df_mahasenti['labels'], 'MahaHate - L3Cube')

F1 Micro MahaHate - L3Cube :  0.8599999999999999
F1 Macro: MahaHate - L3Cube :  0.8599999999999999
Precision: MahaHate - L3Cube :  0.86
Recall: MahaHate - L3Cube :  0.86
Accuracy: MahaHate - L3Cube :  0.86


### Hate Detection - Spanish - Hate = 1, NotHate = 0

In [ ]:
df_real_hate_es = pd.read_csv('/content/hate_es_final.csv')

In [ ]:
df_real_hate_es['Tags'].value_counts()

0    50
1    50
Name: Tags, dtype: int64

In [ ]:
df_real_hate_es['Tags']

0     0
1     1
2     0
3     0
4     1
     ..
95    1
96    1
97    0
98    1
99    1
Name: Tags, Length: 100, dtype: int64

#### ChatGPT

In [ ]:
df['hate_es_ChatGPT'].value_counts()

hate        76
not hate    24
Name: hate_es_ChatGPT, dtype: int64

In [ ]:
label_to_num = {cuisine: i for i, cuisine in enumerate(df['hate_es_ChatGPT'].unique())}      
num_to_label = {i: cuisine for cuisine, i in label_to_num.items()}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'not hate': 0, 'hate': 1}
 
{0: 'not hate', 1: 'hate'}


In [ ]:
df['tokenized_hate_es_ChatGPT'] = df['hate_es_ChatGPT'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_hate_es['Tags'], df['tokenized_hate_es_ChatGPT'], 'ChatGPT')

F1 Micro ChatGPT :  0.68
F1 Macro: ChatGPT :  0.6567996567996568
Precision: ChatGPT :  0.7467105263157895
Recall: ChatGPT :  0.68
Accuracy: ChatGPT :  0.68


#### Perplexity AI

In [ ]:
df['hate_es_Perplexity AI'].value_counts()

hate        71
not hate    29
Name: hate_es_Perplexity AI, dtype: int64

In [ ]:
label_to_num = {cuisine: i for i, cuisine in enumerate(df['hate_es_Perplexity AI'].unique())}      
num_to_label = {i: cuisine for cuisine, i in label_to_num.items()}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'not hate': 0, 'hate': 1}
 
{0: 'not hate', 1: 'hate'}


In [ ]:
df['tokenized_hate_es_Perplexity_AI'] = df['hate_es_Perplexity AI'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_hate_es['Tags'], df['tokenized_hate_es_Perplexity_AI'], 'Perplexity AI')

F1 Micro Perplexity AI :  0.67
F1 Macro: Perplexity AI :  0.6547756041426929
Precision: Perplexity AI :  0.7064108790675084
Recall: Perplexity AI :  0.67
Accuracy: Perplexity AI :  0.67


#### Hate-speech-CNERG/dehatebert-mono-spanish

In [ ]:
df_real_hate_es.head(3)

,Unnamed: 0,Text,Tags
0,0,Un coronel falangista de la Guardia Civil vend...,0
1,1,respuesta de adrenocromo significa que eres ma...,1
2,2,@CabraVoladoraST Si juegas al LoL ya sabes de ...,0


In [ ]:
df_predict_hate_es = df_real_hate_es[['Text']]

In [ ]:
model = ClassificationModel("bert", "Hate-speech-CNERG/dehatebert-mono-spanish")

In [ ]:
predictions, raw_outputs = model.predict(df_predict_hate_es['Text'].tolist())

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [ ]:
df_dehatebert = pd.DataFrame(predictions, columns=['labels'])

In [ ]:
calculate_metrics(df_real_hate_es['Tags'], df_dehatebert['labels'], 'DeHate')

F1 Micro DeHate :  0.66
F1 Macro: DeHate :  0.6458333333333334
Precision: DeHate :  0.6904761904761905
Recall: DeHate :  0.66
Accuracy: DeHate :  0.66


### Sentiment Analysis - Spanish - Positive(1), Neutral(0),  Negative(-1)

In [ ]:
df_real_sentiment_es = pd.read_csv('/content/sentiment_es_final.csv')

In [ ]:
df_real_sentiment_es['Tags'].value_counts()

NEU    33
N      33
P      33
Name: Tags, dtype: int64

In [ ]:
label_to_num = {'N': -1, 'NEU': 0, 'P': 1}
num_to_label = {-1: 'N', 0: 'NEU', 1: 'P'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'N': -1, 'NEU': 0, 'P': 1}
 
{-1: 'N', 0: 'NEU', 1: 'P'}


In [ ]:
df_real_sentiment_es['tokeinzed_Tags'] = df_real_sentiment_es['Tags'].apply(lambda x: label_to_num[x])

In [ ]:
df_real_sentiment_es['tokeinzed_Tags']

0     0
1    -1
2     1
3     1
4     0
     ..
94   -1
95    0
96    1
97    1
98    0
Name: tokeinzed_Tags, Length: 99, dtype: int64

#### ChatGPT

In [ ]:
df['sentiment_es_ChatGPT'].value_counts()

0 negative     1
74 negative    1
72 neutral     1
71 negative    1
70 positive    1
              ..
30 neutral     1
29 positive    1
28 positive    1
27 negative    1
98 neutral     1
Name: sentiment_es_ChatGPT, Length: 99, dtype: int64

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('\d+', '', regex=True)
  df[column_name] = df[column_name].str.replace(' ', '')
  return df

In [ ]:
df = remove_substrings(df, 'sentiment_es_ChatGPT')

In [ ]:
df['sentiment_es_ChatGPT'].unique()

array(['negative', 'positive', 'neutral', nan], dtype=object)

In [ ]:
df = df.dropna(subset=['sentiment_es_ChatGPT'])

In [ ]:
df['sentiment_es_ChatGPT'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [ ]:
label_to_num = {'negative': -1, 'neutral': 0, 'positive': 1}
num_to_label = {-1: 'negative', 0: 'neutral', 1: 'positive'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'negative': -1, 'neutral': 0, 'positive': 1}
 
{-1: 'negative', 0: 'neutral', 1: 'positive'}


In [ ]:
df['tokenized_sentiment_es_ChatGPT'] = df['sentiment_es_ChatGPT'].apply(lambda x: label_to_num[x])

In [ ]:
len(df['tokenized_sentiment_es_ChatGPT'])

99

In [ ]:
calculate_metrics(df_real_sentiment_es['tokeinzed_Tags'], df['tokenized_sentiment_es_ChatGPT'], 'ChatGPT')

F1 Micro ChatGPT :  0.696969696969697
F1 Macro: ChatGPT :  0.6949626865671643
Precision: ChatGPT :  0.6935483870967741
Recall: ChatGPT :  0.696969696969697
Accuracy: ChatGPT :  0.696969696969697


#### Perplexity AI

In [ ]:
df['sentiment_es_Perplexity AI'].value_counts()

neutral     37
positive    31
negative    30
mixed        1
Name: sentiment_es_Perplexity AI, dtype: int64

In [ ]:
df['sentiment_es_Perplexity AI'].unique()

array(['negative', 'positive', 'neutral', 'mixed'], dtype=object)

In [ ]:
def remove_substrings(df, column_name):
  df[column_name] = df[column_name].str.replace('mixed', 'negative')
  return df

In [ ]:
df = remove_substrings(df, 'sentiment_es_Perplexity AI')

In [ ]:
df['sentiment_es_Perplexity AI'].unique()

array(['negative', 'positive', 'neutral'], dtype=object)

In [ ]:
label_to_num = {'negative': -1, 'neutral': 0, 'positive': 1}
num_to_label = {-1: 'negative', 0: 'neutral', 1: 'positive'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'negative': -1, 'neutral': 0, 'positive': 1}
 
{-1: 'negative', 0: 'neutral', 1: 'positive'}


In [ ]:
df['tokenized_sentiment_es_Perplexity_AI'] = df['sentiment_es_Perplexity AI'].apply(lambda x: label_to_num[x])

In [ ]:
len(df['tokenized_sentiment_es_Perplexity_AI'])

99

In [ ]:
calculate_metrics(df_real_sentiment_es['tokeinzed_Tags'], df['tokenized_sentiment_es_Perplexity_AI'], 'Perplexity AI')

F1 Micro Perplexity AI :  0.6565656565656566
F1 Macro: Perplexity AI :  0.6601190476190476
Precision: Perplexity AI :  0.6657948270851496
Recall: Perplexity AI :  0.6565656565656566
Accuracy: Perplexity AI :  0.6565656565656566


#### pysentimiento

In [ ]:
df_real_sentiment_es.head(3)

,Unnamed: 0,Text,Tags,tokeinzed_Tags
0,0,@lantoli podemos usar el término.equipo pepino,NEU,0
1,1,@ISanDesign @Hobby_Consolas jaja y por que lo ...,N,-1
2,2,Cosas que enamora las tostadas calentitas a es...,P,1


In [ ]:
df_predict_sentiment_es = df_real_sentiment_es[['Text']]

In [ ]:
df_predict_sentiment_es.head(3)

,Text
0,@lantoli podemos usar el término.equipo pepino
1,@ISanDesign @Hobby_Consolas jaja y por que lo ...
2,Cosas que enamora las tostadas calentitas a es...


In [ ]:
df_pred = analyzer.predict(df_predict_sentiment_es['Text'].tolist())

Map:   0%|          | 0/99 [00:00<?, ? examples/s]

In [ ]:
str(df_pred[0])[22:25]

'NEU'

In [ ]:
pred_list = list() 
for i in range(len(df_pred)):
  pred_list.append(str(df_pred[i])[22:25])

In [ ]:
print(pred_list)

['NEU', 'NEG', 'POS', 'POS', 'NEU', 'NEG', 'POS', 'NEG', 'NEG', 'POS', 'NEU', 'POS', 'NEG', 'NEU', 'POS', 'NEU', 'NEG', 'POS', 'POS', 'NEG', 'NEU', 'NEG', 'POS', 'NEG', 'POS', 'NEU', 'POS', 'NEG', 'NEG', 'POS', 'POS', 'POS', 'NEG', 'NEG', 'NEU', 'NEG', 'NEU', 'NEG', 'NEU', 'POS', 'POS', 'NEU', 'NEG', 'POS', 'NEG', 'NEG', 'POS', 'NEG', 'POS', 'POS', 'POS', 'NEG', 'NEG', 'NEG', 'NEG', 'POS', 'NEG', 'POS', 'NEG', 'POS', 'POS', 'POS', 'POS', 'POS', 'POS', 'NEU', 'NEG', 'POS', 'NEU', 'NEU', 'POS', 'NEG', 'NEG', 'NEU', 'NEG', 'NEU', 'POS', 'NEG', 'NEG', 'NEG', 'NEU', 'NEG', 'NEG', 'POS', 'POS', 'NEU', 'NEU', 'POS', 'NEU', 'NEG', 'POS', 'NEG', 'NEG', 'NEG', 'NEG', 'POS', 'NEG', 'POS', 'NEU']


In [ ]:
df_prediction = pd.DataFrame(pred_list, columns=['Labels'])

In [ ]:
df_prediction['Labels'].value_counts()

NEG    40
POS    38
NEU    21
Name: Labels, dtype: int64

In [ ]:
label_to_num = {'NEG': -1, 'NEU': 0, 'POS': 1}
num_to_label = {-1: 'NEG', 0: 'NEU', 1: 'POS'}

In [ ]:
print(label_to_num)
print(" ")
print(num_to_label)

{'NEG': -1, 'NEU': 0, 'POS': 1}
 
{-1: 'NEG', 0: 'NEU', 1: 'POS'}


In [ ]:
df_prediction['tokenized_sentiment_es_pysentimiento'] = df_prediction['Labels'].apply(lambda x: label_to_num[x])

In [ ]:
calculate_metrics(df_real_sentiment_es['tokeinzed_Tags'], df_prediction['tokenized_sentiment_es_pysentimiento'], 'pysentimiento')

F1 Micro pysentimiento :  0.7373737373737375
F1 Macro: pysentimiento :  0.722068586046977
Precision: pysentimiento :  0.7337092731829573
Recall: pysentimiento :  0.7373737373737373
Accuracy: pysentimiento :  0.7373737373737373
